# LangChain Playground #1

At its core, LangChain is framework built around LLMs. We have use it to build chatbots, generative question-answering, summarization, and much more.

The core idea of the library is that we can "chain" together different components to create more advanced use cases around LLMs. Chain may consist of multiple components from several modules: Prompt Templates, LLMs, Agents, and Memory.

In [23]:
%pip install -qU openai

Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install -qU langchain

Note: you may need to restart the kernel to use updated packages.


## Using LLMs in LangChain
LangChain support several LLM providers, like HuggingFace and OpenAI.
Let's exmplore the different LLM integrations.

### HuggingFace

In [2]:
%pip install -qU huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [26]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HF_KEY_GOES_HERE"


We can then generate text using a HF Hub model (we'll use `google/flan-t5-x1`) using the Inference API built into Hugging Face Hub.

_(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like `bigscience/bloom-560m` or `google/flan-t5-xxl`)_

In [8]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature": 0.0}
)

template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "Which NFL team won the super bowl in the 2010 season?"
print(llm_chain.run(question))

green bay packers


Asking multiple questions we can done by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template (`"question"`) that is mapped to the question we'd like to ask.

In [11]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 5 ft 10 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}  
]
res = llm_chain.generate(qs)
res

LLMResult(generations=[[Generation(text='green bay packers', generation_info=None)], [Generation(text='190', generation_info=None)], [Generation(text='john glenn', generation_info=None)], [Generation(text='one', generation_info=None)]], llm_output=None)

In [12]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

If I am 6 ft 4 inches, how tall am I in centimeters


### OpenAI

In [29]:
os.environ['OPENAI_API_KEY'] = 'OPENAI_KEY_GOES_HERE'

Then we decide on which model we'd like to use, there are several options but we will go with `text-davinci-003`:

In [27]:
from langchain.llms import OpenAI
davinci = OpenAI(model_name="text-davinci-003")

In [28]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

 The Green Bay Packers won the Super Bowl in the 2010 season.


In [34]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 5 ft 10 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

LLMResult(generations=[[Generation(text=' The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 177.8 centimeters', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' The 12th person to walk on the moon was Harrison Schmitt, an American geologist and NASA astronaut. He was part of the Apollo 17 mission, which took place on December 11, 1972.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have any eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 142, 'completion_tokens': 67, 'prompt_tokens': 75}})

In [1]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))

NameError: name 'llm_chain' is not defined